### CMIP6 Data Issues 
1. The ES-DOC errata pages are used for modelling centers to report issues with their published data here:
      [ES-DOC ERRATA]( https://errata.es-doc.org )
2. A separate list of exceptions is kept as we process the data (concatenating netcdf and saving as zarr)
      [ESGF to GCS Issues]( https://docs.google.com/spreadsheets/d/e/2PACX-1vRxKgz1xCH7zhUoDnl_llgEvbj2ssxoJiTUdbkHkkfWiCKU8EfZtPerar3ELjoIzAda5giR06QvbWGE/pubhtml?gid=128595157&single=true )
3. Issues with the existing Google Cloud collection are crowd sourced here:
      [GCS Issues]( https://tinyurl.com/y5cw76at )

### This notebook updates the list of processing exceptions in Issue 2.

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import datetime
from ast import literal_eval
import os
import numpy as np
import os
gspread.__version__

'3.1.0'

In [2]:
json_keyfile = '/home/naomi/cmip6-zarr/json/Pangeo Hackathon-e48a41b13c91.json'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)

In [3]:
sheet_name = "CMIP6_DataExceptions (Responses)"
sh = gc.open(sheet_name)
print(sh.worksheets())

[<Worksheet 'Form Responses 1' id:252604281>, <Worksheet 'NH_additions' id:128595157>]


In [4]:
wks = sh.worksheet("NH_additions")

In [5]:
data = wks.get_all_values()
headers = data.pop(0)

df_cloud = pd.DataFrame(data, columns=headers)

df_local = pd.read_csv('csv/exceptions.csv')
df_local['name'] = ['/'.join(s[:-2]) for s in df_local.values]

In [6]:
# add new from df_local
for item,row in enumerate(df_local.values):
    name = row[-1]
    #print(item,name)
    df_match = df_cloud[df_cloud.name==name]
    if len(df_match)==0:
        print(item,name,' is not in df_cloud')
        wks.append_row(list(row))
        
# delete old from df
for item,row in enumerate(df_cloud.values):
    name = row[-1]
    #print(item,name)
    df_match = df_local[df_local.name==name]
    if len(df_match)==0:
        print(item,name,' is not in df_local')
        wks.delete_row(item+2)


In [ ]:
assert False  # just scratch space below

In [ ]:
#wks.update_acell('B1', 'Bingo!')
#wks.append_row(['junk','more_junk'])
#wks.add_rows(2)
#for row in df_local.values[:100]:
#    wks.append_row(list(row))

In [7]:
common_cols = df_cloud.columns.tolist()                              #generate list of column names
df12 = pd.merge(df_cloud, df_local, on=common_cols, how='inner')     #extract common rows with merge
df2 = df_local[~df_local['name'].isin(df12['name'])]
df1 = df_cloud[~df_cloud['name'].isin(df12['name'])]
len(df_cloud),len(df_local)

(274, 274)

In [ ]:
#dfdup = df[df.duplicated(keep=False)]

In [ ]:
wks.delete_row(7)

In [ ]:
wks.row_values(8)